In [2]:
import pandas as pd
pd.options.display.max_colwidth = 200

import seaborn as sns
sns.set(rc={'figure.figsize':(15,5)})

import matplotlib.pyplot as plt
import numpy as np

In [3]:
merchant_agg = pd.read_csv("../data/curated/merchant_agg.csv")
merchant_clusters = pd.read_csv("../data/curated/merchant_clusters.csv")

display(merchant_agg.head(3))
display(merchant_clusters.head(3))

,merchant_abn,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,10023283211,2619,154.333333,33301.586223
1,10342410215,729,40.888889,15624.420327
2,10346855916,7,1.400000,2186.800497


,merchant_abn,label
0,10023283211,2
1,10342410215,2
2,10346855916,0


**Merge datasets**

In [4]:
merged_feature_cluster = merchant_clusters.merge(merchant_agg, on='merchant_abn', how='inner').drop('merchant_abn', axis=1)
display(merged_feature_cluster.head(3))

,label,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,2,2619,154.333333,33301.586223
1,2,729,40.888889,15624.420327
2,0,7,1.400000,2186.800497


**Perfrom log transformation**

In [5]:
agg_byCluster = pd.DataFrame(np.log(merged_feature_cluster.iloc[:,1:])).add_prefix('log_')
agg_byCluster['cluster'] = pd.Categorical(merged_feature_cluster.label)

display(agg_byCluster.head(3))
display(agg_byCluster.groupby('cluster').describe().T)

,log_total_number_of_distinct_customers,log_monthly_average_number_of_orders,log_monthly_average_bnpl_revenue,cluster
0,7.870548,5.039115,10.413360,2
1,6.591674,3.710858,9.656590,2
2,1.945910,0.336472,7.690195,0


cluster                                                 0           1  \
log_total_number_of_distinct_customers count  2480.000000  435.000000   
                                       mean      4.254012    9.126523   
                                       std       1.487147    0.529620   
                                       min       0.000000    6.797940   
                                       25%       3.295837    8.799058   
                                       50%       4.454347    9.101529   
                                       75%       5.443498    9.478380   
                                       max       6.680855   10.089137   
log_monthly_average_number_of_orders   count  2480.000000  435.000000   
                                       mean      1.640434    6.578398   
                                       std       1.068996    0.828751   
                                       min       0.000000    3.932914   
                                       25%       0.693147    6.063720   
                                       50%       1.575536    6.434814   
                                       75%       2.556366    6.950386   
                                       max       3.796737    9.529497   
log_monthly_average_bnpl_revenue       count  2480.000000  435.000000   
                                       mean      7.923205   11.170415   
                                       std       0.832944    1.009141   
                                       min       5.122598    8.800315   
                                       25%       7.317890   10.596963   
                                       50%       7.777593   10.971315   
                                       75%       8.625890   11.962458   
                                       max      10.049596   13.056325   

cluster                                                 2  
log_total_number_of_distinct_customers count  1465.000000  
                                       mean      7.146709  
                                       std       0.928573  
                                       min       0.693147  
                                       25%       6.628041  
                                       50%       7.223296  
                                       75%       7.862112  
                                       max       9.808352  
log_monthly_average_number_of_orders   count  1465.000000  
                                       mean      4.299531  
                                       std       0.944823  
                                       min       0.693147  
                                       25%       3.757317  
                                       50%       4.362391  
                                       75%       5.030075  
                                       max       7.535415  
log_monthly_average_bnpl_revenue       count  1465.000000  
                                       mean      9.954474  
                                       std       1.024984  
                                       min       6.480134  
                                       25%       9.276325  
                                       50%      10.217333  
                                       75%      10.752119  
                                       max      12.903122

## Plot the Marginal Distances

In [12]:
def marginal_error(x_variable, agg_byCluster=agg_byCluster):
    '''this function creates plots of marginal distances between each cluster with respect to each feature'''
    sns.scatterplot(data=agg_byCluster, hue='cluster', style='cluster', x=x_variable, y=agg_byCluster['cluster'], palette='deep')

    # compute boundaries for clusters, outliers are removed using 1.5IQR
    description = agg_byCluster.groupby('cluster')[x_variable].describe().T
    description.loc["lb",:] = description.loc["25%"]-1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["ub",:] = description.loc["75%"]+1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["chosen_lower"] = np.where(description.loc["lb",:] < description.loc["min",:], description.loc["min",:], description.loc["lb",:])
    description.loc["chosen_upper"] = np.where(description.loc["ub",:] < description.loc["max",:], description.loc["ub",:], description.loc["max",:])
    
    # visualizing the boundaries
    colour = ['b', 'orange', 'green']
    cluster = [0,1,2]
    for i in range(0,3):
        plt.axvline(x =  description.loc["chosen_lower",cluster[i]], color = colour[i], linestyle = ':')
        plt.axvline(x =  description.loc["chosen_upper",cluster[i]], color = colour[i], linestyle = ':')
        
    # plot the marginal distances
    for i in range(0,2):
        if description.loc["chosen_lower", i+1] < description.loc["chosen_upper",i]:
            plt.axvline(x = description.loc["chosen_upper",i]-(description.loc["chosen_upper",i]-description.loc["chosen_lower", i+1])/2, 
                        color = 'r', label = f'marginal_error_{i,i+1}')
    if description.loc["chosen_lower", 2] < description.loc["chosen_upper",0]:
        plt.axvline(x = description.loc["chosen_upper",0]-(description.loc["chosen_upper",0]-description.loc["chosen_lower", 2])/2, 
                    color = 'r', label = f'marginal_error_{0,2}')
        
    plt.legend()
    plt.savefig(f'../plots/{x_variable}.jpg')
    plt.clf()
    
    return

<Figure size 1500x500 with 0 Axes>

In [ ]:
for x_variable in agg_byCluster.columns[:-1]:
    marginal_error(x_variable)